In [35]:
import os
import numpy as np
import h5py
import json
import torch

from PIL import Image

from collections import Counter
from random import seed, choice, sample

from os import listdir
from tqdm import tqdm
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from pickle import dump

[nltk_data] Downloading package punkt to /Users/evelsve/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [36]:
max_cap_lenght = 60
cap_per_img = 5
min_word_freq = 10


dataset = 'celeb_small_res'
base_filename = dataset +'_' + str(cap_per_img) + '_' + str(min_word_freq)
main_path = '/Users/evelsve/repos/cap'
out_path = main_path+ '/output' + f'/out13'
combine_with_sketches = True


In [37]:
# import shutil

# path_to_images = main_path + '/freakshow_10k'
# for f_name in listdir(path_to_images):
#     file_id = f_name.strip('.jpg').split('_')[2]

#     original = f'{main_path}/freakshow_10k/{f_name}'
#     target = f'{main_path}/freakshow/{file_id}.jpg'

#     shutil.copyfile(original, target)

In [38]:
def load_doc(filename):
    # load doc into memory
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text


def read_create_dataset(directory):
    # two outputs -- path:caption and id:path dictionaries
    paths_caption_dictn, id_path_dictn = dict(), dict()
    path_to_captions = directory+'/captions'
    path_to_images = directory+'/img'
    for f_name in listdir(path_to_images):
        img_filepath = path_to_images + '/' + f_name
        jpgname = f_name.split('.')
        if jpgname[1] == 'jpg':
            image_id = int(jpgname[0])
            cap_filepath = path_to_captions + '/' + str(image_id)+'.txt'
            # add id:path to id_path dict
            id_path_dictn[image_id] = img_filepath
            #load captions
            lines = load_doc(cap_filepath)
            lines = lines.split('\n')
            # add path + list to dictn
            paths_caption_dictn[img_filepath] = list()
            #take only 10 descriptions
            for caption in lines[:10]:
                # add a tokenized caption to path_captions dict
                caption = word_tokenize(caption)
                #caption = nltk.Text(caption)
                caption =[token.lower() for token in caption if token != ',']
                paths_caption_dictn[img_filepath].append(caption)
                
        else: 
            print(f'Encountered random file: {f_name}')
    
    return paths_caption_dictn, id_path_dictn


def dictn_reverser(dictn):
    val_key = {v:k for k, v in dictn.items()}
    return val_key



In [39]:
data, data_id = read_create_dataset(main_path+'/data/'+dataset)

In [40]:
data

{'/Users/evelsve/repos/cap/data/celeb_small_res/img/9733.jpg': [['this',
   'young',
   'woman',
   'has',
   'pointy',
   'nose',
   'and',
   'big',
   'lips',
   '.'],
  ['this',
   'person',
   'has',
   'pointy',
   'nose',
   'and',
   'wears',
   'heavy',
   'makeup',
   '.'],
  ['the',
   'person',
   'is',
   'young',
   'and',
   'has',
   'straight',
   'hair',
   'and',
   'brown',
   'hair',
   '.'],
  ['the',
   'person',
   'is',
   'attractive',
   'and',
   'has',
   'brown',
   'hair',
   'and',
   'pointy',
   'nose',
   '.'],
  ['she',
   'wears',
   'heavy',
   'makeup',
   '.',
   'she',
   'is',
   'attractive',
   'and',
   'has',
   'big',
   'lips',
   '.'],
  ['the',
   'woman',
   'is',
   'young',
   'and',
   'has',
   'big',
   'lips',
   'and',
   'straight',
   'hair',
   '.'],
  ['the',
   'woman',
   'is',
   'wearing',
   'lipstick',
   '.',
   'she',
   'has',
   'brown',
   'hair',
   '.'],
  ['she',
   'is',
   'wearing',
   'lipstick',
   '.',
  

In [41]:
path_data = dict()
path_data['TRAIN'], path_data['TEST'], path_data['VAL']  = dict(), dict(), dict()

for path, captions in data.items():
    # take path, get img name, remove jpg, convert to int
    img_id = int(path.split('/')[-1].strip('.jpg'))
    if img_id < 8000:
        path_data['TRAIN'][path] = captions
    elif 8000 <= img_id < 9000:
        path_data['VAL'][path] = captions
    elif 9000 <= img_id < 10000:
        path_data['TEST'][path] = captions

In [42]:
print(f"In train: {len(path_data['TRAIN'])}\n In test: {len(path_data['TEST'])} \n In val: {len(path_data['VAL'])}")

In train: 8000
 In test: 1000 
 In val: 1000


In [43]:
def create_word_map(data, max_cap_lenght):
    word_freq, captions = dict(), list()
    for path in data:
        # read throuh every tokenized and lowrcased caption as a list of tokens
        for captn in data[path]:
            if len(captn) <= max_cap_lenght:
                pass
            else:
                captn = captn[:max_cap_lenght]
            for w in captn:
                captions.append(w)
                if w in word_freq:
                    word_freq[w] += 1
                else:
                    word_freq[w] = 1
    
    words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq]
    word_map = {k: v + 1 for v, k in enumerate(words)}
    word_map['<unk>'] = len(word_map) + 1
    word_map['<start>'] = len(word_map) + 1
    word_map['<end>'] = len(word_map) + 1
    word_map['<pad>'] = 0
    
    
    return word_map, captions

In [44]:
word_map, all_captions = create_word_map(data, max_cap_lenght)

In [45]:
len(word_map)

69

In [46]:
with open(os.path.join(out_path,'wordmap_'+base_filename+'.json'), 'w') as j:
        json.dump(word_map, j)

In [47]:
sketches_dictn = dict()

if combine_with_sketches:
    sketches_path = "/Users/evelsve/repos/cap/data/sketches"
    for f_name in listdir(sketches_path):
        img_id = int(f_name.split('/')[-1].strip('.jpg'))
        path = sketches_path + '/' + f_name
        sketches_dictn[img_id] = path

In [48]:
len(sketches_dictn)

10000

In [49]:
sketches_dictn[1371]

'/Users/evelsve/repos/cap/data/sketches/1371.jpg'

In [50]:
def create_h5py(directory, path_data, word_map, cap_per_img, max_cap_lenght, sketches_dictn, combine_with_sketches):
    for subset in path_data:
        with h5py.File(os.path.join(directory + '/'+ subset+'_'+base_filename+'.hdf5'), 'w') as h:
            h.attrs['captions_per_image'] = cap_per_img
#             if combine_with_sketches:
            images = h.create_dataset('images', (len(path_data[subset]), 6, 256, 256), dtype='uint8')
#             else:
#                 images = h.create_dataset('images', (len(path_data[subset]), 3, 256, 256), dtype='uint8')
    
            enc_captions = []
            caplens = []
            for i, path in enumerate(tqdm(path_data[subset])):
            # read sketch from folder
                img_id = int(path.split('/')[-1].strip('.jpg'))
                
                imcaps = path_data[subset][path]
                
                
                if len(imcaps) < cap_per_img:
                    captions = imcaps + [choice(imcaps) for _ in range(cap_per_img - len(imcaps))]
                elif len(imcaps) == cap_per_img:
                    captions = imcaps
                else:
                    captions = sample(imcaps, k=cap_per_img)
               
                # Sanity check
                # print(len(captions), cap_per_img, len(captions[0]))
                
                assert len(captions) == cap_per_img
                

                # Read images
                img = Image.open(path)
                img = img.resize((256, 256))
                img = np.transpose(img, (2, 0, 1))
                print('b', img.shape)
                
                if combine_with_sketches:
                    sk_path = sketches_dictn[img_id]
                    print('2',img.shape)
                    
                    sk = Image.open(sk_path)
                    sk = sk.resize((256, 256))
                    sk = np.transpose(sk, (2, 0, 1))
                    print('3',sk.shape)
                    
                    img = torch.from_numpy(img)
                    img = torch.unsqueeze(img, 0)
                    print('4',img.shape)
                    
                    sk = torch.from_numpy(sk)
                    sk = torch.unsqueeze(sk, 0)
                    print('5',sk.shape)
                    
                    img = torch.cat((img, sk), 1)
                    img = torch.squeeze(img, 0)
                    print('6',img.shape)
                    img = img.cpu().detach().numpy()
                    print('6.1',img.shape)
                    img = np.transpose(img, (0, 1, 2))
                    print('7',img.shape)
                    

                assert np.max(img) <= 255

                # Save image to HDF5 file
                images[i] = img

                # encoding the captions
                for j, c in enumerate(captions):
                    enc_c = [word_map['<start>']]
                    for word in c:
                        if word == '.':
                            enc_c.append(word_map['<end>'])
                        else:
                            enc_c.append(word_map.get(word, word_map['<unk>']))
                    
                    additional = [word_map['<pad>'] for i in range((max_cap_lenght - len(c)))]
                    enc_c.extend(additional)
                    

                    # Find caption lengths
                    # because of start/end tokens, it is +2
                    c_len = len(enc_c) + 2

                    enc_captions.append(enc_c)
                    caplens.append(len(c))
            
            # Sanity check
            assert images.shape[0] * cap_per_img == len(enc_captions) == len(caplens)
            

            # Save encoded captions and their lengths to JSON files
            with open(os.path.join(out_path, subset + '_CAPTIONS_' + base_filename + '.json'), 'w') as j:
                json.dump(enc_captions, j)
            with open(os.path.join(out_path, subset + '_CAPLENS_' + base_filename + '.json'), 'w') as j:
                json.dump(caplens, j)

In [51]:
create_h5py(main_path, path_data, word_map, cap_per_img, max_cap_lenght, sketches_dictn, combine_with_sketches)

  0%|▎                                                                                                                                                                   | 14/8000 [00:00<01:07, 117.76it/s]

b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.

  1%|█                                                                                                                                                                   | 50/8000 [00:00<00:49, 160.59it/s]


3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)

  1%|█▊                                                                                                                                                                  | 90/8000 [00:00<00:42, 184.03it/s]

...

 27%|████████████████████████████████████████████                                                                                                                      | 2174/8000 [00:11<00:28, 205.85it/s]

6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])

 28%|████████████████████████████████████████████▋                                                                                                                     | 2204/8000 [00:11<00:30, 189.63it/s]


b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.1 (6, 256, 256)
7 (6, 256, 256)
b (3, 256, 256)
2 (3, 256, 256)
3 (3, 256, 256)
4 torch.Size([1, 3, 256, 256])
5 torch.Size([1, 3, 256, 256])
6 torch.Size([6, 256, 256])
6.

KeyboardInterrupt: 

In [ ]:
import h5py
# filename = "data/out/TRAIN_celeb_5_10.hdf5"

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data = list(f[a_group_key])

In [ ]:
data[200][0]

In [ ]:
def get_dict_from_file(path):
    with open(path, "r") as f:
        dictn = json.load(f)
    return dictn

In [ ]:
data1 = get_dict_from()